In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [10]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.Tanh()
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.Tanh()
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.Tanh()
        self.l4 = nn.Linear(hidden_dim, output_dim)

        self.layers = [self.l1, self.a1,
                       self.l2, self.a2,
                       self.l3, self.a3,
                       self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x

In [ ]:
if __name__ == '__main__':
    np.random.seed(123)
    torch.manual_seed(123)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    '''
    1. データの準備
    '''
    root = os.path.join('~', '.torch', 'mnist')
    transform = transforms.Compose([transforms.ToTensor(),
                                    lambda x: x.view(-1)])
    mnist_train = datasets.MNIST(root=root,
                                 download=True,
                                 train=True,
                                 transform=transform)
    mnist_test = datasets.MNIST(root=root,
                                download=True,
                                train=False,
                                transform=transform)

    train_dataloader = DataLoader(mnist_train,
                                  batch_size=100,
                                  shuffle=True)
    test_dataloader = DataLoader(mnist_test,
                                 batch_size=100,
                                 shuffle=False)

    '''
    2. モデルの構築
    '''
    model = DNN(784, 200, 10).to(device)

    '''
    3. モデルの学習
    '''
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizers.SGD(model.parameters(), lr=0.01)

    def compute_loss(t, y):
        return criterion(y, t)

    def train_step(x, t):
        model.train()
        preds = model(x)
        loss = compute_loss(t, preds)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss, preds

    epochs = 30
    train_loss_history = []
    train_acc_history  = []
    for epoch in range(epochs):
        train_loss = 0.
        train_acc = 0.

        for (x, t) in train_dataloader:
            x, t = x.to(device), t.to(device)
            loss, preds = train_step(x, t)
            train_loss += loss.item()
            train_acc += \
                accuracy_score(t.tolist(),
                               preds.argmax(dim=-1).tolist())

        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader)

        print('epoch: {}, loss: {:.3}, acc: {:.3f}'.format(
            epoch+1,
            train_loss,
            train_acc
        ))
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)

    '''
    4. モデルの評価
    '''
    def test_step(x, t):
        model.eval()
        preds = model(x)
        loss = criterion(preds, t)

        return loss, preds

    test_loss = 0.
    test_acc = 0.

    for (x, t) in test_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = test_step(x, t)
        test_loss += loss.item()
        test_acc += \
            accuracy_score(t.tolist(),
                           preds.argmax(dim=-1).tolist())

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    print('test_loss: {:.3f}, test_acc: {:.3f}'.format(
        test_loss,
        test_acc
    ))

epoch: 1, loss: 1.83, acc: 0.562
epoch: 2, loss: 0.755, acc: 0.809
epoch: 3, loss: 0.487, acc: 0.870
epoch: 4, loss: 0.402, acc: 0.889
epoch: 5, loss: 0.363, acc: 0.897
epoch: 6, loss: 0.339, acc: 0.903
epoch: 7, loss: 0.322, acc: 0.908
epoch: 8, loss: 0.308, acc: 0.912
epoch: 9, loss: 0.297, acc: 0.915
epoch: 10, loss: 0.287, acc: 0.917
epoch: 11, loss: 0.278, acc: 0.919
epoch: 12, loss: 0.27, acc: 0.922
epoch: 13, loss: 0.263, acc: 0.924
epoch: 14, loss: 0.256, acc: 0.926
epoch: 15, loss: 0.249, acc: 0.928
epoch: 16, loss: 0.243, acc: 0.930
epoch: 17, loss: 0.236, acc: 0.931
epoch: 18, loss: 0.23, acc: 0.933
epoch: 19, loss: 0.224, acc: 0.935
epoch: 20, loss: 0.217, acc: 0.937
epoch: 21, loss: 0.211, acc: 0.939
epoch: 22, loss: 0.205, acc: 0.941
epoch: 23, loss: 0.199, acc: 0.942
epoch: 24, loss: 0.193, acc: 0.945
epoch: 25, loss: 0.187, acc: 0.946
epoch: 26, loss: 0.181, acc: 0.948
epoch: 27, loss: 0.176, acc: 0.950
epoch: 28, loss: 0.17, acc: 0.951
epoch: 29, loss: 0.165, acc: 0.95

In [12]:
plt.figure()
plt.plot(range(1, epochs + 1), train_loss_history)
plt.ylabel('Loss')
plt.xlabel('Epoch')

ax2 = plt.gca().twinx()
ax2.plot(range(1, epochs + 1), train_acc_history,
        '--')
ax2.set_ylabel('Accuracy')
lines, labels = plt.gca().get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines + lines2, labels + labels2, loc='upper left')

plt.title('MNIST Training — Loss & Accuracy')
plt.tight_layout()
plt.savefig('mnist_training_curve.png')
plt.close()
print('Training curve saved as mnist_training_curve.png')


Training curve saved as mnist_training_curve.png
